Authors:
Rajeswari Nakka, G.V.S.N.R.V.Prasad, R.Kiran Kumar

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import heapq
import joblib
from sklearn.model_selection import train_test_split

## Loading Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CF Project/Datasets/Final 3 Datasets (Full Length)/reviews.csv')
df = df.rename(columns={'RecipeId': 'itemId', 'AuthorId': 'userId'})

d = (dict(enumerate(df['userId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['userId'] = df['userId'].map(new_d)

d = (dict(enumerate(df['itemId'].unique())))
new_d = {}
new_d = {value:key for key, value in d.items()}
df['itemId'] = df['itemId'].map(new_d)

train, test = train_test_split(df[(df['userId']<1500) & (df['itemId']<800)], test_size=0.2)

In [ ]:
train_dataframe = train
train_dataframe

,ReviewId,itemId,userId,AuthorName,Rating,Review,DateSubmitted,DateModified
2233,4382,221,1262,Brattee,5,Loved this receipe....I love different type of...,2001-10-15T13:30:18Z,2001-10-15T13:30:18Z
375,536,299,305,Amy Bradley,4,Funny story . . I went to buy the ingredients...,2001-03-16T12:15:50Z,2001-03-16T12:15:50Z
1865,3858,526,716,Lisa Barnes,5,This was the most amazing Shepherd's Pie I hav...,2001-09-24T14:17:14Z,2001-09-24T14:17:14Z
1070,1774,611,740,Ann Arber,0,"No, no base. No springform pan. Either make it...",2001-07-24T13:39:36Z,2001-07-24T13:39:36Z
632,936,482,503,LC Haxton,0,love it!!!!,2001-05-29T16:36:24Z,2001-05-29T16:36:24Z
...,...,...,...,...,...,...,...,...
1301,2355,737,864,wcsdmatron,0,"Haven't yet tried the recipe; however, I want ...",2001-08-15T18:18:56Z,2001-08-15T18:18:56Z
2167,4288,689,1025,Klynn Fisher,5,Wonderful cake.It is also very good with dates...,2001-10-11T13:43:56Z,2001-10-11T13:43:56Z
469603,507720,642,510,Bergy,5,I really enjoyed this recipe - The ribs were f...,2007-10-31T22:18:31Z,2007-10-31T22:18:31Z
2472,4685,719,451,Mark H.,5,Excellent recipe. The gravy is fantastic. We c...,2001-10-24T13:57:27Z,2001-10-24T13:57:27Z


In [ ]:
test_dataframe = test
test_dataframe

,ReviewId,itemId,userId,AuthorName,Rating,Review,DateSubmitted,DateModified
2722,4987,193,1376,Carol,5,I made these pancakes this morning for the fam...,2001-11-05T12:17:01Z,2001-11-05T12:17:01Z
5949,8690,390,765,Kat2355,4,"I didn't have the opportunity to use ""real"" mo...",2002-02-18T13:12:44Z,2002-02-18T13:12:44Z
3145,5517,206,1428,RecipeNut,5,I thought this recipe was great! I did change...,2001-11-23T15:34:21Z,2001-11-23T15:34:21Z
6652,9459,661,940,sheila,4,"this is a great recipe,we use to make these ab...",2002-03-04T13:43:31Z,2002-03-04T13:43:31Z
787,1241,585,587,Joanne Johnson,5,This is a great recipie! I have been using th...,2001-06-18T15:49:16Z,2001-06-18T15:49:16Z
...,...,...,...,...,...,...,...,...
1331,2400,3,883,Patricia Black,0,"This recipe sounds delicious, and I'm definite...",2001-08-21T14:01:39Z,2001-08-21T14:01:39Z
433,625,336,355,D Martin,5,This soup was delicious. I did not add a blan...,2001-04-03T15:24:28Z,2001-04-03T15:24:28Z
142,224,128,107,mark hoover,5,this is a traditional dish my family has enjoy...,2000-12-25T15:13:43Z,2000-12-25T15:13:43Z
74902,81843,569,1192,Lennie,5,What better way to start the new year but with...,2004-01-01T10:41:40Z,2004-01-01T10:41:40Z


TODO: Split the data in test and train

In [ ]:
'''
CREATE PIVOT TABLE
'''
pivot_table = train_dataframe.pivot_table(values='Rating',columns='itemId',index='userId')
# REPLACE NULL VALUES WITH ZEROES
pivot_table_nn = pivot_table.fillna(0)
pivot_table_nn

itemId,0,1,2,3,4,5,6,7,8,10,12,13,15,16,17,18,19,20,21,22,23,24,25,26,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,...,757,759,760,761,762,763,764,765,766,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Algorithm** Adaptive Similarity Measure Model
</br>
Input: user-item rating matrix</br>
Output: user-movie rating matrix with predicted ratings</br>

In [ ]:
''' 
FOR ALL ITEMS IN THE DATASET M
COMPUTE THE INVERSE USER FREQUENCY(IUF) FOR EACH ITEM m in M
'''
iuf_dict = {} # will store the item ids as keys and values as iuf value caculated
for item in pivot_table_nn.columns:
  item_ratings = pivot_table_nn[item]
  filtered_item_ratings = [x for x in item_ratings if x > 0.0]
  Um = len(filtered_item_ratings)
  U = len(item_ratings)
  try:
    iuf = math.log(Um/U)
  except:
    iuf = 0
  iuf_dict[item] = iuf

SHould i drop 0s when finidng similarities in all of the similarity functions

In [ ]:
'''
FUCTION TO CALCULATE SIMILARITY_ASM
'''
def sim_asm(user_id1, user_id2):
  avg_u1_rating = np.array(pivot_table_nn.loc[user_id1]).mean()
  avg_u2_rating = np.array(pivot_table_nn.loc[user_id2]).mean()
  numerator = 0.0
  denominator = 0.0
  for item in pivot_table_nn.columns:
    iuf = iuf_dict[item]
    num_subterm1 = pivot_table_nn[item][user_id1] - avg_u1_rating
    num_subterm2 = pivot_table_nn[item][user_id2] - avg_u2_rating
    numerator += ((iuf**2) * num_subterm1 * num_subterm2)
  denominator = abs(numerator)**0.5
  print('numerator,denominator',numerator,denominator)
  if denominator==0:
    return 0
  similarity = numerator/denominator
  return similarity


call above and below function for each pair of user

In [ ]:
'''
FUNCTION TO CALCULATE JACCARD SIMILARITY
'''
def sim_jaccard(user_id1, user_id2):
  numerator=0
  denominator=0
  for item in pivot_table_nn.columns:
    if pivot_table_nn[item][user_id1]>0 and pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      numerator += 1 
    if pivot_table_nn[item][user_id1]>0 or pivot_table_nn[item][user_id2]>0:
      #if ratings for both users available for given movie increase numerator by 1
      denominator += 1 
  if denominator==0:
    return 0
  similarity = 1 - (numerator/denominator)
  return similarity

In [ ]:
'''
FUNCTION TO CALCULATE MODIFIED SIMILARITY
'''
def modified_sim(user_id1, user_id2):
  term1 = sim_asm(user_id1, user_id2)
  term2 = sim_jaccard(user_id1, user_id2)
  print(term1,term2)
  modified_simialrity = term1 * term2
  print(modified_simialrity)
  return modified_simialrity

In [ ]:

#DICTIONARY
i=0
dict_users = {}
for user in pivot_table_nn.index:
  dict_users[user] = i
  i+=1

In [ ]:
'''
CREATE A SIMILARITY MATRIX OF DIMENSION (NO. OF USERS *  NO. OF USERS)
'''


similarity_matrix = np.zeros([pivot_table_nn.shape[0],pivot_table_nn.shape[0]])
i=0
for user1 in pivot_table_nn.index:
  for user2 in pivot_table_nn.index:
    i+=1
    print("Cell Number:",i)
    if  similarity_matrix[dict_users[user1]][dict_users[user2]] == 0:
      similarity_matrix[dict_users[user1]][dict_users[user2]] = modified_sim(user1,user2)
      similarity_matrix[dict_users[user2]][dict_users[user1]] = similarity_matrix[dict_users[user1]][dict_users[user2]] 
similarity_matrix

Streaming output truncated to the last 5000 lines.
0 1.0
0.0
Cell Number: 129898
numerator,denominator -2.078576852334602 1.4417270380812734
-1.4417270380812737 1.0
-1.4417270380812737
Cell Number: 129899
numerator,denominator -0.7271503750121137 0.8527311270336704
-0.8527311270336704 1.0
-0.8527311270336704
Cell Number: 129900
numerator,denominator -0.6870608585907045 0.8288913430545071
-0.8288913430545071 1.0
-0.8288913430545071
Cell Number: 129901
numerator,denominator -2.167637044919304 1.4722897285926109
-1.4722897285926109 1.0
-1.4722897285926109
Cell Number: 129902
numerator,denominator -1.276787355656943 1.129950156271038
-1.129950156271038 1.0
-1.129950156271038
Cell Number: 129903
numerator,denominator -2.6679276917375874 1.6333792247171468
-1.6333792247171466 1.0
-1.6333792247171466
Cell Number: 129904
numerator,denominator -0.9160811447876039 0.9571212800829391
-0.9571212800829392 1.0
-0.9571212800829392
Cell Number: 129905
numerator,denominator -0.8990792727763286 0.948197

In [ ]:
#CREATE DATAFRAME
import pickle
similarity_matrix_df = pd.DataFrame(similarity_matrix)
# similarity_matrix_df['original_user_id'] = list(dict_users.keys())
similarity_matrix_df

In [ ]:
#SAVE SIMILARITY MATRIX
file_path = '/content/drive/MyDrive/CF Project/Datasets/similarity_matrix_df_real_reviews'
file_object = open(file_path,'wb')
pickle.dump(similarity_matrix_df,file_object)


In [ ]:
def find_delu(mu, user):
  # FIND DEL_U
  count_items_rated_by_user = len([x for x in pivot_table_nn.loc[user] if x > 0])
  term1 = 1 / count_items_rated_by_user
  summation = 0
  for r in pivot_table_nn.loc[user]:
    if r > 0:
      summation += (r-mu)
  del_u = term1 * summation
  return del_u
def find_deli(mu, user, del_u):
  # FIND DEL_I
  count_users_rated_item = len([x for x in pivot_table_nn[item]])
  term1 = 1 / count_users_rated_item
  summation = 0
  for r in pivot_table_nn[item]:
    if r > 0:
      summation += (r - del_u - mu)
  del_i = summation
  return del_i
def find_delui(mu,user):
  # FIND DEL_UI
  del_u = find_delu(mu,user)
  del_i = find_deli(mu,user,del_u)
  del_ui = mu + del_u + del_i
  return del_ui
def find_key(val, my_dict):
  for key, value in my_dict.items():
    if val == value:
      return key
  return "key doesn't exist"

In [ ]:
'''
PREDICT THE RATINGS
'''
# FIND MU, GLOBAL AVERAGE
mu = train_dataframe['Rating'].mean()
N = 5 # NUMBER OF NEIGHBOURS TO TAKE
mae = []
pred = []
actuals = []
for index,row in test_dataframe.iterrows():
  user = row['user_id']
  item = row['recipe_id']
  given_rating = row['rating']
  #GET NEIGHBOUR 'N' USERS
  if user not in dict_users.keys() or item not in pivot_table_nn.columns or dict_users[user] not in similarity_matrix_df.index:
    #cold start problem
    continue
  the_row = similarity_matrix_df[dict_users[user]]
  the_row = the_row.sort_values(ascending=False)
  # print('the_row',the_row[:N])
  neighbour_index = the_row.index
  neighbour_similarities = list(the_row)
  
  # FIND PRED_UI
  num = 0# how to the user_ids here
  deno = 0# how to get 
  # iterate over neighbour users
  the_delui = find_delui(mu,user)
  for i in range(N):
    deno += abs(neighbour_similarities[i])
    real_neighbour_id = find_key(neighbour_index[i],dict_users)
    rating_neigh = pivot_table_nn[item][real_neighbour_id]
    delui_neigh = find_delui(mu,real_neighbour_id)
    num += ((neighbour_similarities[i]) *(rating_neigh-delui_neigh))
  if deno>0:
    term2 = num/deno
  else:
    term2 = 0
  pred_ui = the_delui + term2
  mae.append(abs(pred_ui-given_rating))
  pred.append(pred_ui)
  actuals.append(given_rating)

# print("MAE:",np.array(mae).mean())
# print("Prediction:",np.array(pred))
# print("Actual:",np.array(actuals))

normalized = [((x-min(pred)) / (max(pred) - min(pred)) * 5)+3 for x in pred]
print(np.mean(abs(np.array(normalized)-np.array(actuals))))

